## Create Directory

In [ ]:
#!create a data directory
%mkdir data

## Connecting to LA Socrata Database

In [1]:
#Import Dependencies
import pandas as pd
from sodapy import Socrata

In [ ]:
#Import user credentials
from config import *

In [ ]:
client = Socrata('data.lacity.org', App_Token, username=username,password=password)

In [ ]:
results = client.get("k8cc-2d49", limit=500000)

In [ ]:
#Store the data in a csv format for reusability
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df.to_csv("data/collision.csv", index=False)

## Data Munging

In [2]:
#Read the stored collision data
collisionDF = pd.read_csv('data/collision.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Print Sample data
collisionDF[:5]

,area,area_name,crm_cd,crm_cd_desc,cross_street,date_occ,date_rptd,dr_no,location,location_1,mocodes,premis_cd,premis_desc,rpt_dist_no,time_occ,vict_age,vict_descent,vict_sex
0,1,Central,997,TRAFFIC COLLISION,2ND,2010-04-20T00:00:00.000,NaN,100109637,ALAMEDA,"{'type': 'Point', 'coordinates': [-118.2381, 3...",NaN,101.0,STREET,138,1235,55.0,B,M
1,7,Wilshire,997,TRAFFIC COLLISION,RIMPAU,2010-12-21T00:00:00.000,NaN,100719446,PICO,"{'type': 'Point', 'coordinates': [-118.337, 34...",NaN,101.0,STREET,758,2218,37.0,B,M
2,1,Central,997,TRAFFIC COLLISION,BROADWAY,2010-06-04T00:00:00.000,NaN,100111904,5TH,"{'type': 'Point', 'coordinates': [-118.2507, 3...",NaN,101.0,STREET,153,2310,29.0,H,M
3,1,Central,997,TRAFFIC COLLISION,SAN PEDRO,2010-03-22T00:00:00.000,NaN,100108154,4TH,"{'type': 'Point', 'coordinates': [-118.2422, 3...",NaN,101.0,STREET,148,1325,43.0,B,F
4,13,Newton,997,TRAFFIC COLLISION,ADAMS,2010-01-12T00:00:00.000,NaN,101304785,HILL,"{'type': 'Point', 'coordinates': [-118.2708, 3...",NaN,101.0,STREET,1321,1800,34.0,O,M


In [4]:
#Only needed columns are save in the new Collision DataFrame
collisionDF= pd.DataFrame({'age': collisionDF.vict_age,
                          'descent': collisionDF.vict_descent, 
                          'sex':collisionDF.vict_sex,
                          'area':collisionDF.area_name, 
                          'location':collisionDF.location, 
                          'intersection': collisionDF.cross_street,
                          'geojson':collisionDF.location_1, 
                          'date':collisionDF.date_occ, 
                          'time':collisionDF.time_occ,
                          'reporting_district':collisionDF.rpt_dist_no
                         })

In [6]:
#Format the date in the Date column
collisionDF.date = pd.to_datetime(collisionDF.date)
collisionDF[:10]

,age,area,date,descent,geojson,intersection,location,reporting_district,sex,time
0,55.0,Central,2010-04-20,B,"{'type': 'Point', 'coordinates': [-118.2381, 3...",2ND,ALAMEDA,138,M,1235
1,37.0,Wilshire,2010-12-21,B,"{'type': 'Point', 'coordinates': [-118.337, 34...",RIMPAU,PICO,758,M,2218
2,29.0,Central,2010-06-04,H,"{'type': 'Point', 'coordinates': [-118.2507, 3...",BROADWAY,5TH,153,M,2310
3,43.0,Central,2010-03-22,B,"{'type': 'Point', 'coordinates': [-118.2422, 3...",SAN PEDRO,4TH,148,F,1325
4,34.0,Newton,2010-01-12,O,"{'type': 'Point', 'coordinates': [-118.2708, 3...",ADAMS,HILL,1321,M,1800
5,77.0,West LA,2010-09-20,B,"{'type': 'Point', 'coordinates': [-118.3918, 3...",CARDIFF,PICO,857,F,1635
6,74.0,Harbor,2010-11-19,H,"{'type': 'Point', 'coordinates': [-118.3097, 3...",9TH,WEYMOUTH,561,F,1529
7,22.0,Mission,2010-07-25,H,"{'type': 'Point', 'coordinates': [-118.4721, 3...",CABRITO,ORION,1991,M,800
8,54.0,Hollywood,2010-10-14,W,"{'type': 'Point', 'coordinates': [-118.3753, 3...",MULHOLLAND DR,LAUREL CANYON BL,621,M,1030
9,NaN,Olympic,2010-08-31,H,"{'type': 'Point', 'coordinates': [-118.3091, 3...",WESTERN AV,OLYMPIC BL,2053,M,1000


In [7]:
#Added a new column - (Day of the Week) based on the Date column
collisionDF['day_of_week'] = collisionDF['date'].dt.weekday_name

In [8]:
#Reorganize the columns so that its more readable
collisionDF = collisionDF[['age', 'descent', 'sex','area', 'location', 'intersection',
                           'geojson', 'date', 'day_of_week', 'time','reporting_district']]

In [9]:
#Print a sample data
collisionDF[:5]

,age,descent,sex,area,location,intersection,geojson,date,day_of_week,time,reporting_district
0,55.0,B,M,Central,ALAMEDA,2ND,"{'type': 'Point', 'coordinates': [-118.2381, 3...",2010-04-20,Tuesday,1235,138
1,37.0,B,M,Wilshire,PICO,RIMPAU,"{'type': 'Point', 'coordinates': [-118.337, 34...",2010-12-21,Tuesday,2218,758
2,29.0,H,M,Central,5TH,BROADWAY,"{'type': 'Point', 'coordinates': [-118.2507, 3...",2010-06-04,Friday,2310,153
3,43.0,B,F,Central,4TH,SAN PEDRO,"{'type': 'Point', 'coordinates': [-118.2422, 3...",2010-03-22,Monday,1325,148
4,34.0,O,M,Newton,HILL,ADAMS,"{'type': 'Point', 'coordinates': [-118.2708, 3...",2010-01-12,Tuesday,1800,1321


In [10]:
#Check the count ofevery columns
collisionDF.count()

age                   382396
descent               412457
sex                   412902
area                  418509
location              418509
intersection          400211
geojson               418509
date                  418509
day_of_week           418509
time                  418509
reporting_district    418509
dtype: int64

In [11]:
#Drop records entries for columns with NaN values
# Note that after resetting the index, the gap is gone
cleanDF = collisionDF.dropna(subset = ['age', 'descent', 'sex']).reset_index(drop=True)

In [12]:
#Check the count of records per column
cleanDF.count()

age                   379407
descent               379407
sex                   379407
area                  379407
location              379407
intersection          364867
geojson               379407
date                  379407
day_of_week           379407
time                  379407
reporting_district    379407
dtype: int64

In [13]:
#Save the cleaned collision data to a csv file
cleanDF.to_csv("data/Cleaned_collision.csv", index=False)

## Database Creation

In [14]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [15]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///la_colission.sqlite")

In [16]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [17]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'collision'

    id = Column(Integer, primary_key=True)
    age = Column(Integer)
    descent = Column(Text)
    sex = Column(Text)
    area = Column(Text)
    location = Column(Text)
    intersection= Column(Text)
    geojson= Column(Text)
    date = Column(Text)
    day_of_week = Column(Text)
    time = Column(Text)
    reporting_district = Column(Text)
   
    def __repr__(self):
        return f"id={self.id}, age={self.age}, descent={self.descent}, sex={self.sex}, area={self.area}, location={self.location}, intersection={self.intersection}, geojson={self.geojson}, date={self.date}, day_of_week={self.day_of_week}, time={self.time}, reporting_district={self.reporting_district}"

In [18]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [19]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv('data/Cleaned_collision.csv')

In [20]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = new_df.to_dict(orient='records')

In [21]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

[{'age': 55.0, 'descent': 'B', 'sex': 'M', 'area': 'Central', 'location': 'ALAMEDA', 'intersection': '2ND', 'geojson': "{'type': 'Point', 'coordinates': [-118.2381, 34.0455]}", 'date': '2010-04-20', 'day_of_week': 'Tuesday', 'time': 1235, 'reporting_district': 138}, {'age': 37.0, 'descent': 'B', 'sex': 'M', 'area': 'Wilshire', 'location': 'PICO', 'intersection': 'RIMPAU', 'geojson': "{'type': 'Point', 'coordinates': [-118.337, 34.0488]}", 'date': '2010-12-21', 'day_of_week': 'Tuesday', 'time': 2218, 'reporting_district': 758}, {'age': 29.0, 'descent': 'H', 'sex': 'M', 'area': 'Central', 'location': '5TH', 'intersection': 'BROADWAY', 'geojson': "{'type': 'Point', 'coordinates': [-118.2507, 34.0481]}", 'date': '2010-06-04', 'day_of_week': 'Friday', 'time': 2310, 'reporting_district': 153}, {'age': 43.0, 'descent': 'B', 'sex': 'F', 'area': 'Central', 'location': '4TH', 'intersection': 'SAN PEDRO', 'geojson': "{'type': 'Point', 'coordinates': [-118.2422, 34.0454]}", 'date': '2010-03-22', '

In [22]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [23]:
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table('collision', metadata, autoload=True)

In [24]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [25]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [26]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from collision limit 5").fetchall()

[(1, 55, 'B', 'M', 'Central', 'ALAMEDA', '2ND', "{'type': 'Point', 'coordinates': [-118.2381, 34.0455]}", '2010-04-20', 'Tuesday', '1235', '138'),
 (2, 37, 'B', 'M', 'Wilshire', 'PICO', 'RIMPAU', "{'type': 'Point', 'coordinates': [-118.337, 34.0488]}", '2010-12-21', 'Tuesday', '2218', '758'),
 (3, 29, 'H', 'M', 'Central', '5TH', 'BROADWAY', "{'type': 'Point', 'coordinates': [-118.2507, 34.0481]}", '2010-06-04', 'Friday', '2310', '153'),
 (4, 43, 'B', 'F', 'Central', '4TH', 'SAN PEDRO', "{'type': 'Point', 'coordinates': [-118.2422, 34.0454]}", '2010-03-22', 'Monday', '1325', '148'),
 (5, 34, 'O', 'M', 'Newton', 'HILL', 'ADAMS', "{'type': 'Point', 'coordinates': [-118.2708, 34.0258]}", '2010-01-12', 'Tuesday', '1800', '1321')]